<a href="https://colab.research.google.com/github/hikmatfarhat-ndu/CSC645/blob/master/IMDB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [54]:
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.datasets import imdb
import numpy as np


## The data

THe movie review dataset is a set of 50000 reviews of movies (half training, half test). Each review contains a set of words and is labeled positive (1) or negative (0). For convenience each word index refer to its frequency of occurence in the dataset. For example a word with index 5 is the fifth most frequently used data set. The indices 0,1 and 2 are reserved so 5 really means the third most frequent.

Details about the dataset can be found here [Keras IMDB](https://keras.io/api/datasets/imdb/)

In this exercise we choose only the first 10000 most frequent words to be included. Any word that is not among them is given the index 2.

First we load data set without omitting any words

### Data details

We would like to have an idea about the number of reviews, the average length of a review. Also we compute how many entries with values 0,1,2 and 3. The number 0 is used for padding and 1 to denote the beginning of each sequence. The number 2 is used for missing words. Finally, the number 3 is never used since as you will see later we will shift the indices by 3.

In [34]:
(x_train,y_train),(x_test,y_test)=tf.keras.datasets.imdb.load_data()
print("The number of reviews in the x_train data set = {}\n".format(x_train.shape[0]))
print("The average length of reviews = {}".format(np.mean([len(x) for x in x_train])))
print("With standard deviation = {}".format(np.std([len(x) for x in x_train])))
print("The number of 0's in the x_train data set = {}\n".format(sum([1 for x in np.hstack(x_train) if x==0])))
print("The number of 1's in the x_train data set = {}\n".format(sum([1 for x in np.hstack(x_train) if x==1])))
print("The number of 2's in the x_train data set = {}\n".format(sum([1 for x in np.hstack(x_train) if x==2])))
print("The number of 3's in the x_train data set = {}\n".format(sum([1 for x in np.hstack(x_train) if x==3])))

The number of reviews in the x_train data set = 25000

The average length of reviews = 238.71364
With standard deviation = 176.49367364852034
The number of 0's in the x_train data set = 0

The number of 1's in the x_train data set = 25000

The number of 2's in the x_train data set = 1

The number of 3's in the x_train data set = 0



Now when we choose only the first 5000 most frequent words and compute the number of 2's in the data set. As you can see the number of 2's is now very large since all the "ignored" words were given the code 2.

In [55]:
(x_train,y_train),(x_test,y_test)=imdb.load_data(num_words=10000)
print("The number of 2's in the x_train data set = {}\n".format(sum([1 for x in np.hstack(x_train) if x==2])))

The number of 2's in the x_train data set = 341611



### Word index

Keras provides also a dictionary of word to index. From that we build a dictionary of index to words. We use the index_to_word to display the first review in the data set.

In [56]:

word_to_index=imdb.get_word_index()
print(word_to_index['the'])
print(word_to_index['and'])
index_to_word=dict([(key,val) for (val,key) in word_to_index.items()])
review = " ".join( [index_to_word.get(i - 3, "***") for i in x_train[0]] )
print(review)

x_train=pad_sequences(x_train,maxlen=500)
print(x_train[0])
print(len(x_train[0]))











1
2
*** this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert *** is an amazing actor and now the same being director *** father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for *** and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also *** to the two little boy's that played the *** of norman and paul they were just brilliant children are often left out of the *** list i think because the stars that play them all grown up are such a big profile for the whole film but these children are amazing and should be praised for what they have 

In [ ]:
a=list(x.keys())
b=list(x.values())
w=a[b.index(1408)]
for i in range(1,20):
  print(y[i])
str=" ".join([y.get(i) for i in x_train[1]])
print(len(np.unique(np.hstack(x_train))))


the
and
a
of
to
is
br
in
it
i
this
that
was
as
for
with
movie
but
film
9998
